In [27]:
import pandas as pd
pd.set_option('display.max_rows', 999)

# 讀取 CSV 檔案
df = pd.read_feather('Step1.feather')

# 計算總筆數
total_rows = len(df)

# 找出有缺失值的欄位及相關統計
missing_info = df.isnull().sum()
missing_info = missing_info[missing_info > 0]  # 只保留有缺失的欄位

# 組成結果 DataFrame，包含 欄位名稱、缺失數量、缺失占比（百分比）
missing_df = pd.DataFrame({
    'Column': missing_info.index,
    'Missing Count': missing_info.values,
    'Missing Ratio (%)': (missing_info.values / total_rows * 100).round(2)
})

# 依照缺失比例排序（可選）
missing_df = missing_df.sort_values(by='Missing Ratio (%)', ascending=False)

# 顯示結果
missing_df.head(100)


,Column,Missing Count,Missing Ratio (%)
300,日外資_外資自營商賣張,20000,100.00
362,月營收_預估年營收(千),20000,100.00
490,季IFRS財報_財務信評,20000,100.00
383,月營收_重要子公司本年累計營收淨額(千),20000,100.00
382,月營收_重要子公司本月營業收入淨額(千),20000,100.00
363,月營收_累計營收達成率(%),20000,100.00
299,日外資_外資自營商買張,20000,100.00
301,日外資_外資自營商買賣超,20000,100.00
306,日外資_與前日異動原因,19752,98.76
311,日投信_投信賣均價,18401,92.01


In [28]:
def suggest_missing_strategy(ratio):
    if ratio < 5:
        return "可忽略或簡單填補"
    elif ratio < 30:
        return "建議填補（平均/中位/類別）"
    elif ratio < 60:
        return "慎重填補或考慮刪除"
    elif ratio < 90:
        return "傾向刪除欄位"
    else:
        return "建議刪除欄位"

In [29]:
# 填補策略處理器
def fill_missing_values(df):
    total_rows = len(df)
    missing_info = df.isnull().sum()
    missing_info = missing_info[missing_info > 0]

    report = []

    for col in missing_info.index:
        missing_count = missing_info[col]
        missing_ratio = missing_count / total_rows * 100
        col_type = df[col].dtype

        strategy = suggest_missing_strategy(missing_ratio)

        if missing_ratio < 5:
            if pd.api.types.is_numeric_dtype(col_type):
                df[col].fillna(df[col].median(), inplace=True)
                fill_type = "中位數"
            else:
                df[col].fillna(df[col].mode().dropna().iloc[0], inplace=True)
                fill_type = "眾數"
        elif missing_ratio < 30:
            if pd.api.types.is_numeric_dtype(col_type):
                df[col].fillna(df[col].mean(), inplace=True)
                fill_type = "平均數"
            else:
                df[col].fillna(df[col].mode().dropna().iloc[0], inplace=True)
                fill_type = "眾數"
        else:
            fill_type = "未處理"

        report.append({
            'Column': col,
            'Missing Count': missing_count,
            'Missing Ratio (%)': round(missing_ratio, 2),
            'Strategy': strategy,
            'Fill Method': fill_type
        })

    return df, pd.DataFrame(report)

In [30]:
# ✨ 自動處理缺失值
cleaned_df, report_df = fill_missing_values(df)

# 📊 顯示處理報告
print("\n✅ 缺失值處理報告：\n")
report_df

C:\Users\TMP-214\AppData\Local\Temp\ipykernel_52760\1736873653.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\TMP-214\AppData\Local\Temp\ipykernel_52760\1736873653.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For


✅ 缺失值處理報告：



,Column,Missing Count,Missing Ratio (%),Strategy,Fill Method
0,外資券商_分點買賣力,6039,30.20,慎重填補或考慮刪除,未處理
1,外資券商_分點成交力(%),247,1.23,可忽略或簡單填補,中位數
2,外資券商_分點吃貨比(%),247,1.23,可忽略或簡單填補,中位數
3,外資券商_分點出貨比(%),247,1.23,可忽略或簡單填補,中位數
4,外資券商_前1天分點買賣力,5972,29.86,建議填補（平均/中位/類別）,平均數
...,...,...,...,...,...
9936,賣超第15名分點前20天賣均張,3441,17.21,建議填補（平均/中位/類別）,平均數
9937,賣超第15名分點前20天買均價,3441,17.21,建議填補（平均/中位/類別）,平均數
9938,賣超第15名分點前20天賣均價,3441,17.21,建議填補（平均/中位/類別）,平均數
9939,賣超第15名分點前20天買均值(千),3441,17.21,建議填補（平均/中位/類別）,平均數


In [31]:
# 計算總筆數
total_rows = len(df)

# 找出有缺失值的欄位及相關統計
missing_info = df.isnull().sum()
missing_info = missing_info[missing_info > 0]  # 只保留有缺失的欄位

# 組成結果 DataFrame，包含 欄位名稱、缺失數量、缺失占比（百分比）
missing_df = pd.DataFrame({
    'Column': missing_info.index,
    'Missing Count': missing_info.values,
    'Missing Ratio (%)': (missing_info.values / total_rows * 100).round(2)
})

# 依照缺失比例排序（可選）
missing_df = missing_df.sort_values(by='Missing Ratio (%)', ascending=False)

# 顯示結果
missing_df.head(100)

,Column,Missing Count,Missing Ratio (%)
36,季IFRS財報_財務信評,20000,100.00
12,月營收_預估年營收(千),20000,100.00
15,月營收_重要子公司本年累計營收淨額(千),20000,100.00
3,日外資_外資自營商買張,20000,100.00
4,日外資_外資自營商賣張,20000,100.00
5,日外資_外資自營商買賣超,20000,100.00
14,月營收_重要子公司本月營業收入淨額(千),20000,100.00
13,月營收_累計營收達成率(%),20000,100.00
6,日外資_與前日異動原因,19752,98.76
10,日投信_投信賣均價,18401,92.01


In [32]:
#30%以上	不分類型	不處理或考慮刪除
print(df.shape)
df = df.dropna(axis=1)
print(df.shape)
print(df.isnull().sum().sum())

(20000, 10214)
(20000, 10177)
0


In [33]:
# 記錄原本資料列數
original_rows = len(df)

# 刪除重複資料（保留第一筆）
df = df.drop_duplicates(keep='first')

# 計算刪除數量
removed_rows = original_rows - len(df)

print(f"✅ 已刪除 {removed_rows} 筆重複資料。")

✅ 已刪除 0 筆重複資料。


In [34]:
df.to_feather('Step2.feather')